# Pulse compilation and Optimization 

basis_gates = A collection of classes for 1 and 2 qubit basis gates, gate-set objects = (I,X,Y,Z,CX,CZ)<br>

<br>
Operation = Takes a circuit instruction in the form of string and returns its numpy matrix rep. This additional feature has been added for verifying the matrix representation of the complete pulse train, before and after compilation/optimization.<br>



random_two_qubit_circuit_instructions = A class that generates a string of one and two qubit gates from the gate-set.<br>
<br>
random_one_qubit_circuit_instructions = A class that generates a string of one qubit gate from the gate-set.<br>
<br>
circuit = A class that compiles a pulse train of one or two qubit gates to the basis gates (I,X,Z,CX).

In [41]:
from basis_gates import I,X,Y,Z,CX
from Operation import Operation
from Moment import Moment
from random_two_qubit_circuit_instructions import random_two_qubit_circuit_instructions
from random_one_qubit_circuit_instructions import random_one_qubit_circuit_instructions
from circuit import circuit
import numpy

In [42]:
#This function maps the circuit instruction seq. to a unitary matrix
def circuit_to_matrix(circ_str):
    list_of_ops=circ_str.split(';\n')[1:]
    Op_matrix=numpy.eye(4)+0*1j
    for i in range(len(list_of_ops)):
        decomposed_ops=list_of_ops[i].split('|')
        for j in range(len(decomposed_ops)):
            M=Operation(decomposed_ops[j]).matrix
            Op_matrix=Op_matrix@M
    return Op_matrix        

# Pulse compilation for one qubit circuit instructions.

In [43]:
#Take as input the depth
depth=7 
#Construct a random one quantum circuit instruction string from the basis gates
random_circ_str=random_one_qubit_circuit_instructions(int(depth)).circ_str
#circuit class is initialized with this circuit string
circ=circuit(random_circ_str)
'''
circuit string is converted to moment string where quantum operations
are implemented simulatenously whenever possible
'''
moments,instruction_str=circ.circuit_to_moments()


print("____________circuit instructions___________\n",random_circ_str,'\n',"____________circuit instructions after first compilation___________\n",instruction_str)

____________circuit instructions___________
 qubits 2;
Y(1.6291789594) [0]|I [1];
Z(1.7254370363) [0]|I [1];
X(-2.9730280269) [0]|I [1];
Z(1.6143494254) [0]|I [1];
Y(0.5845019928) [0]|I [1];
Z(2.0751475080) [0]|I [1];
Z(-1.1342033299) [0]|I [1] 
 ____________circuit instructions after first compilation___________
 qubits 2;
Y(1.6291789594) [0]|I [1];
Z(1.7254370363) [0]|I [1];
X(-2.9730280269) [0]|I [1];
Z(1.6143494254) [0]|I [1];
Y(0.5845019928) [0]|I [1];
Z(0.9409442) [0]|I [1];



### Translate from __[X,Y,Z,CX]__ to __[X,Z,CX]__

In [44]:
#translate to the required gate-set as mentioned in the technical test
choice='3' #compiles to gate-set ['X','Z','CX'] other two choices '1' and '2' allows compiling to other gate sets ['X','Y','Z','CZ'] and ['X','Z','Z','CX']
circ=circuit(instruction_str[:-2])
translated_circ=circ.compile_to_gateset(instruction_str[:-2],choice)
circ=circuit(translated_circ)
moments,instruction_str_trans=circ.circuit_to_moments()
instruction_str_trans=instruction_str_trans[:-4]
moments=moments[:-5]
print("________translated circuit moments___________\n",moments)

________translated circuit moments___________
 qubits 2;
0|X(0.78539816339) [0]|I [1];
1|Z(1.6291789594) [0]|I [1];
2|X(-0.78539816339) [0]|I [1];
3|Z(1.7254370363) [0]|I [1];
4|X(-2.9730280269) [0]|I [1];
5|Z(1.6143494254) [0]|I [1];
6|X(0.78539816339) [0]|I [1];
7|Z(0.5845019928) [0]|I [1];
8|X(-0.78539816339) [0]|I [1];
9|Z(0.9409442) [0]|I [1];



In [45]:
# check equivalence between unitary matrix forms of original and compiled circuit
numpy.allclose(circuit_to_matrix(random_circ_str),circuit_to_matrix(instruction_str_trans))

True

# Pulse compilation for two qubit circuit instructions.

In [46]:
#Take as input the depth
depth=7 
#Construct a random two quantum circuit instruction string from the basis gates
random_circ_str=random_two_qubit_circuit_instructions(int(depth)).circ_str
#circuit class is initialized with this circuit string
circ=circuit(random_circ_str)
'''
circuit string is converted to moment string where quantum operations
are implemented simulatenously whenever possible
'''
moments,instruction_str=circ.circuit_to_moments()


print("____________circuit instructions___________\n",random_circ_str,'\n',"____________circuit instructions after first compilation___________\n",instruction_str)

____________circuit instructions___________
 qubits 2;
Y(-2.1382685232) [0]|I [1];
I [0]|X(-2.3966840438) [1];
X(0.1242241135) [0]|I [1];
I [0]|Z(0.8714701783) [1];
X(-1.9071639737) [0]|I [1];
CX [0,1];
I [0]|Y(1.8971277542) [1] 
 ____________circuit instructions after first compilation___________
 qubits 2;
Y(-2.1382685232) [0]|X(-2.3966840438) [1];
X(-1.7829399) [0]|Z(0.8714701783) [1];
CX [0,1];
I [0]|Y(1.8971277542) [1];



In [47]:
#translate to the required gate-set as mentioned in the technical test
choice='3' #compiles to gate-set ['X','Z','CX'] other two choices '1' and '2' allows compiling to other gate sets ['X','Y','Z','CZ'] and ['X','Z','Z','CX']
circ=circuit(instruction_str[:-2])
translated_circ=circ.compile_to_gateset(instruction_str[:-2],choice)
circ=circuit(translated_circ)
moments,instruction_str_trans=circ.circuit_to_moments()
instruction_str_trans=instruction_str_trans[:-4]
moments=moments[:-5]
print("________translated circuit moments___________\n",moments)

________translated circuit moments___________
 qubits 2;
0|X(0.78539816339) [0]|I [1];
1|Z(-2.1382685232) [0]|I [1];
2|X(-2.5683381) [0]|X(-2.3966840438) [1];
3|I [0]|Z(0.8714701783) [1];
4|CX [0,1];
5|I [0]|X(0.78539816339) [1];
6|I [0]|Z(1.8971277542) [1];
7|I [0]|X(-0.78539816339) [1];


In [48]:
# check equivalence between unitary matrix forms of original and compiled circuit
numpy.allclose(circuit_to_matrix(random_circ_str),circuit_to_matrix(instruction_str_trans))

True